In [1]:
import pandas as pd
from datetime import datetime
import os
import numpy as np
import pymongo

In [18]:
file = pd.read_excel(os.path.join("Original_Files","CountyHealthRankingsData","2020","2020 County Health Rankings Texas Data - v1_0.xlsx"),sheet_name='Ranked Measure Data',header=(0,1))
df = pd.DataFrame(file)
df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Premature death  \
                FIPS              State             County          Deaths   
0              48000              Texas                NaN        297317.0   
1              48001              Texas           Anderson          1130.0   
2              48003              Texas            Andrews           180.0   
3              48005              Texas           Angelina          1375.0   
4              48007              Texas            Aransas           492.0   

                                                                          \
  Years of Potential Life Lost Rate 95% CI - Low 95% CI - High   Z-Score   
0                       6650.619481  6617.683571   6683.555391       NaN   
1                      10927.038043  9999.281173  11854.794914  1.402356   
2                       7137.509901  5773.784378   8501.235425 -0.736774   
3                       8913.812861  8220.517694   9607.108027  0.265922   
4                       8770.440897  7435.439854  10105.441941  0.184990   

                                                  ...  \
  YPLL Rate (AIAN) YPLL Rate (AIAN) 95% CI - Low  ...   
0              NaN                           NaN  ...   
1              NaN                           NaN  ...   
2              NaN                           NaN  ...   
3              NaN                           NaN  ...   
4              NaN                           NaN  ...   

                  Driving alone to work  \
  % Drive Alone (Hispanic) 95% CI - Low   
0                                   NaN   
1                             75.344910   
2                             67.270440   
3                             75.227855   
4                             63.736360   

                                                                \
  % Drive Alone (Hispanic) 95% CI - High % Drive Alone (White)   
0                                    NaN                   NaN   
1                              93.750788             88.398544   
2                              80.765108             86.001725   
3                              85.226559             84.833274   
4                              85.233420             80.556668   

                                                                          \
  % Drive Alone (White) 95% CI - Low % Drive Alone (White) 95% CI - High   
0                                NaN                                 NaN   
1                          87.058797                           89.738291   
2                          80.886002                           91.117447   
3                          82.856492                           86.810055   
4                          75.967120                           85.146216   

  Long commute - driving alone                                             \
     # Workers who Drive Alone % Long Commute - Drives Alone 95% CI - Low   
0                     12842820                          38.2    37.961702   
1                        18980                          26.0    22.100716   
2                         8184                          30.8    24.228420   
3                        35176                          16.3    14.229412   
4                         9765                          28.9    21.557099   

                           
  95% CI - High   Z-Score  
0     38.438298       NaN  
1     29.899284 -0.344842  
2     37.371580  0.025791  
3     18.370588 -1.093831  
4     36.242901 -0.120918  

[5 rows x 240 columns]

In [19]:
df.columns = df.columns.map('_'.join)
df.head()

,Unnamed: 0_level_0_FIPS,Unnamed: 1_level_0_State,Unnamed: 2_level_0_County,Premature death_Deaths,Premature death_Years of Potential Life Lost Rate,Premature death_95% CI - Low,Premature death_95% CI - High,Premature death_Z-Score,Premature death_YPLL Rate (AIAN),Premature death_YPLL Rate (AIAN) 95% CI - Low,...,Driving alone to work_% Drive Alone (Hispanic) 95% CI - Low,Driving alone to work_% Drive Alone (Hispanic) 95% CI - High,Driving alone to work_% Drive Alone (White),Driving alone to work_% Drive Alone (White) 95% CI - Low,Driving alone to work_% Drive Alone (White) 95% CI - High,Long commute - driving alone_# Workers who Drive Alone,Long commute - driving alone_% Long Commute - Drives Alone,Long commute - driving alone_95% CI - Low,Long commute - driving alone_95% CI - High,Long commute - driving alone_Z-Score
0,48000,Texas,NaN,297317.0,6650.619481,6617.683571,6683.555391,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12842820,38.2,37.961702,38.438298,NaN
1,48001,Texas,Anderson,1130.0,10927.038043,9999.281173,11854.794914,1.402356,NaN,NaN,...,75.344910,93.750788,88.398544,87.058797,89.738291,18980,26.0,22.100716,29.899284,-0.344842
2,48003,Texas,Andrews,180.0,7137.509901,5773.784378,8501.235425,-0.736774,NaN,NaN,...,67.270440,80.765108,86.001725,80.886002,91.117447,8184,30.8,24.228420,37.371580,0.025791
3,48005,Texas,Angelina,1375.0,8913.812861,8220.517694,9607.108027,0.265922,NaN,NaN,...,75.227855,85.226559,84.833274,82.856492,86.810055,35176,16.3,14.229412,18.370588,-1.093831
4,48007,Texas,Aransas,492.0,8770.440897,7435.439854,10105.441941,0.184990,NaN,NaN,...,63.736360,85.233420,80.556668,75.967120,85.146216,9765,28.9,21.557099,36.242901,-0.120918


In [20]:
file2 = pd.read_excel(os.path.join("Original_Files","CountyHealthRankingsData","2020","2020 County Health Rankings Texas Data - v1_0.xlsx"),sheet_name='Additional Measure Data',header=(0,1))
df2 = pd.DataFrame(file2)
df2.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Life expectancy  \
                FIPS              State             County Life Expectancy   
0              48000              Texas                NaN       79.087770   
1              48001              Texas           Anderson       73.351755   
2              48003              Texas            Andrews       77.459564   
3              48005              Texas           Angelina       76.130326   
4              48007              Texas            Aransas       77.876650   

                                                     \
  95% CI - Low 95% CI - High Life Expectancy (AIAN)   
0    79.051708     79.123832                    NaN   
1    72.625007     74.078503                    NaN   
2    76.120559     78.798568                    NaN   
3    75.522938     76.737714                    NaN   
4    76.704959     79.048341                    NaN   

                                                                            \
  Life Expectancy (AIAN) 95% CI - Low Life Expectancy (AIAN) 95% CI - High   
0                                 NaN                                  NaN   
1                                 NaN                                  NaN   
2                                 NaN                                  NaN   
3                                 NaN                                  NaN   
4                                 NaN                                  NaN   

                           ... Demographics                       \
  Life Expectancy (Asian)  ...   % Hispanic # Non-Hispanic White   
0                     NaN  ...    39.610168             11912849   
1                     NaN  ...    17.965103                34059   
2                     NaN  ...    56.641659                 7229   
3                     NaN  ...    22.492307                52523   
4                     NaN  ...    27.429388                16063   

                                                    \
  % Non-Hispanic White # Not Proficient in English   
0            41.505516                     1957944   
1            58.664760                        1388   
2            39.877538                        1306   
3            60.307491                        3586   
4            67.514291                         571   

                                                                              \
  % Not Proficient in English 95% CI - Low 95% CI - High   % Female  # Rural   
0                    7.563623     7.490636      7.636610  50.316413  3847522   
1                    2.529523     1.721252      3.337795  38.773964    39204   
2                    8.028524     5.400509     10.656539  48.841571     2440   
3                    4.394823     3.739092      5.050555  51.214807    37383   
4                    2.424011     0.855062      3.992960  50.609449     6313   

              
     % Rural  
0  15.300999  
1  67.063533  
2  16.502097  
3  43.082366  
4  27.260558  

[5 rows x 270 columns]

In [23]:
list(df.columns.values)

['Unnamed: 0_level_0_FIPS',
 'Unnamed: 1_level_0_State',
 'Unnamed: 2_level_0_County',
 'Premature death_Deaths',
 'Premature death_Years of Potential Life Lost Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Premature death_YPLL Rate (AIAN)',
 'Premature death_YPLL Rate (AIAN) 95% CI - Low',
 'Premature death_YPLL Rate (AIAN) 95% CI - High',
 'Premature death_YPLL Rate (Asian)',
 'Premature death_YPLL Rate (Asian) 95% CI - Low',
 'Premature death_YPLL Rate (Asian) 95% CI - High',
 'Premature death_YPLL Rate (Black)',
 'Premature death_YPLL Rate (Black) 95% CI - Low',
 'Premature death_YPLL Rate (Black) 95% CI - High',
 'Premature death_YPLL Rate (Hispanic)',
 'Premature death_YPLL Rate (Hispanic) 95% CI - Low',
 'Premature death_YPLL Rate (Hispanic) 95% CI - High',
 'Premature death_YPLL Rate (White)',
 'Premature death_YPLL Rate (White) 95% CI - Low',
 'Premature death_YPLL Rate (White) 95% CI - High',
 'Poor or fair health_% Fa

In [24]:
df.rename(columns = {'Unnamed: 0_level_0_FIPS' : 'FIPS',
 'Unnamed: 1_level_0_State' :'State',
 'Unnamed: 2_level_0_County':'County',
 'Premature death_Deaths':'Premature_Deaths',
 'Premature death_Years of Potential Life Lost Rate':'Yrs_Potential_Life_Lost_per100K(YPLL)',
 'Poor or fair health_% Fair or Poor Health':'%_Fair/Poor_Health',
 'Poor physical health days_Average Number of Physically Unhealthy Days':'Avg_Days_Poor_Physical_Health_per_Month',
 'Poor mental health days_Average Number of Mentally Unhealthy Days':'Avg_Days_Mentally_Unhealthy_per_Month',
 'Low birthweight_Unreliable':'Low_BirthWt_Unreliable',
 'Low birthweight_% Low Birthweight':'%_Low_Birthweight_Births',
 'Adult smoking_% Smokers':'%_Adult_Smokers',
 'Adult obesity_% Adults with Obesity':'%_Adult_Obesity',
 'Food environment index_Food Environment Index':'Healthy_Food_Access(0=Worst,10=Best)',
 'Physical inactivity_% Physically Inactive':'%_Adults_Physical_Inactivity',
 'Access to exercise opportunities_% With Access to Exercise Opportunities':'%_Access_to_Excercise_Opportunities',
 'Excessive drinking_% Excessive Drinking':'%_Adults_Excessive_Drinking',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths':'#_Alcohol-Impaired_Motor_Vehicle_Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths':'#_Motor_Vehicle_Deaths',
 'Alcohol-impaired driving deaths_% Driving Deaths with Alcohol Involvement':'%_Driving_Deaths_with_Alcohol_Involvement',
 'Sexually transmitted infections_# Chlamydia Cases':'#_Chlamydia_Cases',
 'Sexually transmitted infections_Chlamydia Rate':'Chlamydia_Cases_per_100K',
 'Teen births_Teen Birth Rate':'Teen_Birth_Rate',
 'Uninsured_# Uninsured':'#_Ppl_Under65_w/o_Insurance',
 'Uninsured_% Uninsured':'%_Ppl_Under65_w/o_Insurance',
 'Primary care physicians_# Primary Care Physicians':'#_Primary_Care_Physicians(PCP)_in_Pt_Care',
 'Primary care physicians_Primary Care Physicians Rate':'PCP_per_100K',
 'Primary care physicians_Primary Care Physicians Ratio':'Population_to_PCP_ratio',
 'Dentists_# Dentists':'#_Dentists',
 'Dentists_Dentist Rate':'Dentists_per_100K',
 'Dentists_Dentist Ratio':'Population_to_Dentists_ratio',
 'Mental health providers_# Mental Health Providers':'#_Mental_Health_Providers(MHP)',
 'Mental health providers_Mental Health Provider Rate':'MHP_per_100K',
 'Mental health providers_Mental Health Provider Ratio':'Population_to_MHP_ratio',
 'Preventable hospital stays_Preventable Hospitalization Rate':'Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)',
 'Mammography screening_% With Annual Mammogram':'%_Female_Medicare_Enrollees(65-74)_with_Annual_Mammogram',
 'High school graduation_Cohort Size':'#_Students_Expected_to_Graduate',
 'High school graduation_High School Graduation Rate':'Graduation_Rate',
 'Some college_# Some College':'#_Adults(25-44)_with_Some_Post-secondary_Ed',
 'Some college_Population':'Adults(25-44)',
 'Some college_% Some College':'%_Adults(25-44)_with_Some_Post-secondary_Ed',
 'Unemployment_# Unemployed':'#_Ppl(16+)_Unemployed_&_Looking_for_Work',
 'Unemployment_Labor Force':'Size_of_Labor_Force',
 'Unemployment_% Unemployed':'%_Ppl(16+)_Unemployed_&_Looking_for_Work',
 'Children in poverty_% Children in Poverty':'%_Children(under18)_Living_in_Poverty',
 'Income inequality_80th Percentile Income':'80th_Percentile_MHI',
 'Income inequality_20th Percentile Income':'20th_Percentile_MHI',
 'Income inequality_Income Ratio':'Ratio_of_Household_Income_80th_to_20th',
 'Children in single-parent households_# Single-Parent Households':'#_Children_Living_in_Single_Parent_Household',
 'Children in single-parent households_# Households':'#_Children_in_Households',
 'Children in single-parent households_% Single-Parent Households':'%_Children_Living_in_Single_Parent_Household',
 'Social associations_# Associations':'#_Associations',
 'Social associations_Social Association Rate':'Associations_per_10K',
 'Violent crime_Annual Average Violent Crimes':'Annual_Avg_Violent_Crimes',
 'Violent crime_Violent Crime Rate':'Violen_Crimes_per_100K',
 'Injury deaths_# Injury Deaths':'#_Injury_Deaths',
 'Injury deaths_Injury Death Rate':'Injury_Death_Rate_per_100K',
 'Air pollution - particulate matter_Average Daily PM2.5':'Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter',
 'Drinking water violations_Presence of Water Violation':'County_Affected_by_Water_Violation(1=Yes,0=No)',
 'Severe housing problems_% Severe Housing Problems':'%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)',
 'Severe housing problems_Severe Housing Cost Burden':'%_Households_with_High_Housing_Costs',
 'Severe housing problems_Overcrowding':'%_Households_with_Overcrowding',
 'Severe housing problems_Inadequate Facilities':'%_Households_Lacking_Kitchen_or_Plumbing_Facilities',
 'Driving alone to work_% Drive Alone to Work':'%_Workers_Who_Drive_Alone_to_Work',
 'Long commute - driving alone_# Workers who Drive Alone':'#_Workers_who_Commute(car,truck,van)_Alone',
 'Long commute - driving alone_% Long Commute - Drives Alone':'%_Lone_Commuters_who_Commute_More_Than_30Min'}, 
                  inplace = True)

In [26]:
df = df.drop(['Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_YPLL Rate (AIAN) 95% CI - Low',
 'Premature death_YPLL Rate (AIAN) 95% CI - High',
 'Premature death_YPLL Rate (Asian) 95% CI - Low',
 'Premature death_YPLL Rate (Asian) 95% CI - High',
 'Premature death_YPLL Rate (Black) 95% CI - Low',
 'Premature death_YPLL Rate (Black) 95% CI - High',
 'Premature death_YPLL Rate (Hispanic) 95% CI - Low',
 'Premature death_YPLL Rate (Hispanic) 95% CI - High',
 'Premature death_YPLL Rate (White) 95% CI - Low',
 'Premature death_YPLL Rate (White) 95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_% LBW (AIAN) 95% CI - Low',
 'Low birthweight_% LBW (AIAN) 95% CI - High',
 'Low birthweight_% LBW (Asian) 95% CI - Low',
 'Low birthweight_% LBW (Asian) 95% CI - High',
 'Low birthweight_% LBW (Black) 95% CI - Low',
 'Low birthweight_% LBW (Black) 95% CI - High',
 'Low birthweight_% LBW (Hispanic) 95% CI - Low',
 'Low birthweight_% LBW (Hispanic) 95% CI - High',
 'Low birthweight_% LBW (White) 95% CI - Low',
 'Low birthweight_% LBW (White) 95% CI - High',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Alcohol-impaired driving deaths_95% CI - Low',
 'Alcohol-impaired driving deaths_95% CI - High',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Teen Birth Rate (AIAN) 95% CI - Low',
 'Teen births_Teen Birth Rate (AIAN) 95% CI - High',
 'Teen births_Teen Birth Rate (Asian) 95% CI - Low',
 'Teen births_Teen Birth Rate (Asian) 95% CI - High',
 'Teen births_Teen Birth Rate (Black) 95% CI - Low',
 'Teen births_Teen Birth Rate (Black) 95% CI - High',
 'Teen births_Teen Birth Rate (Hispanic) 95% CI - Low',
 'Teen births_Teen Birth Rate (Hispanic) 95% CI - High',
 'Teen births_Teen Birth Rate (White) 95% CI - Low',
 'Teen births_Teen Birth Rate (White) 95% CI - High',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Injury Death Rate (AIAN) 95% CI - Low',
 'Injury deaths_Injury Death Rate (AIAN) 95% CI - High',
 'Injury deaths_Injury Death Rate (Asian) 95% CI - Low',
 'Injury deaths_Injury Death Rate (Asian) 95% CI - High',
 'Injury deaths_Injury Death Rate (Black) 95% CI - Low',
 'Injury deaths_Injury Death Rate (Black) 95% CI - High',
 'Injury deaths_Injury Death Rate (Hispanic) 95% CI - Low',
 'Injury deaths_Injury Death Rate (Hispanic) 95% CI - High',
 'Injury deaths_Injury Death Rate (White) 95% CI - Low',
 'Injury deaths_Injury Death Rate (White) 95% CI - High',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Severe Housing Cost Burden 95% CI - Low',
 'Severe housing problems_Severe Housing Cost Burden 95% CI - High',
 'Severe housing problems_Overcrowding 95% CI - Low',
 'Severe housing problems_Overcrowding 95% CI - High',
 'Severe housing problems_Inadequate Facilities 95% CI - Low',
 'Severe housing problems_Inadequate Facilities 95% CI - High',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_% Drive Alone (AIAN) 95% CI - Low',
 'Driving alone to work_% Drive Alone (AIAN) 95% CI - High',
 'Driving alone to work_% Drive Alone (Asian) 95% CI - Low',
 'Driving alone to work_% Drive Alone (Asian) 95% CI - High',
 'Driving alone to work_% Drive Alone (Black) 95% CI - Low',
 'Driving alone to work_% Drive Alone (Black) 95% CI - High',
 'Driving alone to work_% Drive Alone (Hispanic) 95% CI - Low',
 'Driving alone to work_% Drive Alone (Hispanic) 95% CI - High',
 'Driving alone to work_% Drive Alone (White) 95% CI - Low',
 'Driving alone to work_% Drive Alone (White) 95% CI - High',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Z-Score',
 'Low birthweight_Z-Score',
 'Adult smoking_Z-Score',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Z-Score',
 'Uninsured_Z-Score',
 'Primary care physicians_Z-Score',
 'Dentists_Z-Score',
 'Mental health providers_Z-Score',
 'Preventable hospital stays_Z-Score',
 'Mammography screening_Z-Score',
 'Flu vaccinations_Z-Score',
 'High school graduation_Z-Score',
 'Some college_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_Z-Score',
 'Income inequality_Z-Score',
 'Children in single-parent households_Z-Score',
 'Social associations_Z-Score',
 'Violent crime_Z-Score',
 'Injury deaths_Z-Score',
 'Drinking water violations_Z-Score',
 'Severe housing problems_Z-Score',
 'Driving alone to work_Z-Score',
 'Long commute - driving alone_Z-Score',
 'Premature death_YPLL Rate (AIAN)',
 'Premature death_YPLL Rate (Asian)',
 'Premature death_YPLL Rate (Black)',
 'Premature death_YPLL Rate (Hispanic)',
 'Premature death_YPLL Rate (White)',
 'Low birthweight_% LBW (AIAN)',
 'Low birthweight_% LBW (Asian)',
 'Low birthweight_% LBW (Black)',
 'Low birthweight_% LBW (Hispanic)',
 'Low birthweight_% LBW (White)',
 'Teen births_Teen Birth Rate (AIAN)',
 'Teen births_Teen Birth Rate (Asian)',
 'Teen births_Teen Birth Rate (Black)',
 'Teen births_Teen Birth Rate (Hispanic)',
 'Teen births_Teen Birth Rate (White)',
 'Preventable hospital stays_Preventable Hosp. Rate (AIAN)',
 'Preventable hospital stays_Preventable Hosp. Rate (Asian)',
 'Preventable hospital stays_Preventable Hosp. Rate (Black)',
 'Preventable hospital stays_Preventable Hosp. Rate (Hispanic)',
 'Preventable hospital stays_Preventable Hosp. Rate (White)',
 'Mammography screening_% Screened (AIAN)',
 'Mammography screening_% Screened (Asian)',
 'Mammography screening_% Screened (Black)',
 'Mammography screening_% Screened (Hispanic)',
 'Mammography screening_% Screened (White)',
 'Flu vaccinations_% Vaccinated (AIAN)',
 'Flu vaccinations_% Vaccinated (Asian)',
 'Flu vaccinations_% Vaccinated (Black)',
 'Flu vaccinations_% Vaccinated (Hispanic)',
 'Flu vaccinations_% Vaccinated (White)',
 'Children in poverty_% Children in Poverty (AIAN)',
 'Children in poverty_% Children in Poverty (Asian)',
 'Children in poverty_% Children in Poverty (Black)',
 'Children in poverty_% Children in Poverty (Hispanic)',
 'Children in poverty_% Children in Poverty (White)',
 'Injury deaths_Injury Death Rate (AIAN)',
 'Injury deaths_Injury Death Rate (Asian)',
 'Injury deaths_Injury Death Rate (Black)',
 'Injury deaths_Injury Death Rate (Hispanic)',
 'Injury deaths_Injury Death Rate (White)',
 'Driving alone to work_% Drive Alone (AIAN)',
 'Driving alone to work_% Drive Alone (Asian)',
 'Driving alone to work_% Drive Alone (Black)',
 'Driving alone to work_% Drive Alone (Hispanic)',
 'Driving alone to work_% Drive Alone (White)',
              'Flu vaccinations_% Vaccinated',
              'Air pollution - particulate matter_Z-Score',]
                             ,axis=1)

In [27]:
df.head()

,FIPS,State,County,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,Injury_Death_Rate_per_100K,Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter,"County_Affected_by_Water_Violation(1=Yes,0=No)","%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)",%_Households_with_High_Housing_Costs,%_Households_with_Overcrowding,%_Households_Lacking_Kitchen_or_Plumbing_Facilities,%_Workers_Who_Drive_Alone_to_Work,"#_Workers_who_Commute(car,truck,van)_Alone",%_Lone_Commuters_who_Commute_More_Than_30Min
0,48000,Texas,NaN,297317.0,6650.619481,20.806624,3.716692,3.813413,NaN,8.328265,...,56.581312,8.75,NaN,17.667800,NaN,NaN,NaN,80.557105,12842820,38.2
1,48001,Texas,Anderson,1130.0,10927.038043,19.727723,3.867507,3.971116,NaN,7.307041,...,82.427383,9.90,Yes,14.423657,9.332113,4.526252,0.965600,86.406744,18980,26.0
2,48003,Texas,Andrews,180.0,7137.509901,19.851002,3.649546,3.731177,NaN,6.292750,...,85.209436,7.30,Yes,16.326531,7.211268,7.977737,1.391466,78.763441,8184,30.8
3,48005,Texas,Angelina,1375.0,8913.812861,20.900430,4.091402,4.334904,NaN,9.050049,...,68.612902,9.80,Yes,17.461879,12.337038,4.902443,1.770782,80.819309,35176,16.3
4,48007,Texas,Aransas,492.0,8770.440897,20.550541,4.116061,4.255922,NaN,9.263521,...,98.080649,9.30,Yes,13.821990,11.233596,2.565445,0.418848,78.535586,9765,28.9


In [21]:
df2.columns = df2.columns.map('_'.join)
df2.head()

,Unnamed: 0_level_0_FIPS,Unnamed: 1_level_0_State,Unnamed: 2_level_0_County,Life expectancy_Life Expectancy,Life expectancy_95% CI - Low,Life expectancy_95% CI - High,Life expectancy_Life Expectancy (AIAN),Life expectancy_Life Expectancy (AIAN) 95% CI - Low,Life expectancy_Life Expectancy (AIAN) 95% CI - High,Life expectancy_Life Expectancy (Asian),...,Demographics_% Hispanic,Demographics_# Non-Hispanic White,Demographics_% Non-Hispanic White,Demographics_# Not Proficient in English,Demographics_% Not Proficient in English,Demographics_95% CI - Low,Demographics_95% CI - High,Demographics_% Female,Demographics_# Rural,Demographics_% Rural
0,48000,Texas,NaN,79.087770,79.051708,79.123832,NaN,NaN,NaN,NaN,...,39.610168,11912849,41.505516,1957944,7.563623,7.490636,7.636610,50.316413,3847522,15.300999
1,48001,Texas,Anderson,73.351755,72.625007,74.078503,NaN,NaN,NaN,NaN,...,17.965103,34059,58.664760,1388,2.529523,1.721252,3.337795,38.773964,39204,67.063533
2,48003,Texas,Andrews,77.459564,76.120559,78.798568,NaN,NaN,NaN,NaN,...,56.641659,7229,39.877538,1306,8.028524,5.400509,10.656539,48.841571,2440,16.502097
3,48005,Texas,Angelina,76.130326,75.522938,76.737714,NaN,NaN,NaN,NaN,...,22.492307,52523,60.307491,3586,4.394823,3.739092,5.050555,51.214807,37383,43.082366
4,48007,Texas,Aransas,77.876650,76.704959,79.048341,NaN,NaN,NaN,NaN,...,27.429388,16063,67.514291,571,2.424011,0.855062,3.992960,50.609449,6313,27.260558


In [28]:
list(df2.columns.values)

['Unnamed: 0_level_0_FIPS',
 'Unnamed: 1_level_0_State',
 'Unnamed: 2_level_0_County',
 'Life expectancy_Life Expectancy',
 'Life expectancy_95% CI - Low',
 'Life expectancy_95% CI - High',
 'Life expectancy_Life Expectancy (AIAN)',
 'Life expectancy_Life Expectancy (AIAN) 95% CI - Low',
 'Life expectancy_Life Expectancy (AIAN) 95% CI - High',
 'Life expectancy_Life Expectancy (Asian)',
 'Life expectancy_Life Expectancy (Asian) 95% CI - Low',
 'Life expectancy_Life Expectancy (Asian) 95% CI - High',
 'Life expectancy_Life Expectancy (Black)',
 'Life expectancy_Life Expectancy (Black) 95% CI - Low',
 'Life expectancy_Life Expectancy (Black) 95% CI - High',
 'Life expectancy_Life Expectancy (Hispanic)',
 'Life expectancy_Life Expectancy (Hispanic) 95% CI - Low',
 'Life expectancy_Life Expectancy (Hispanic) 95% CI - High',
 'Life expectancy_Life Expectancy (White)',
 'Life expectancy_Life Expectancy (White) 95% CI - Low',
 'Life expectancy_Life Expectancy (White) 95% CI - High',
 'Prematu

In [34]:
df2 = df2.drop(['Life expectancy_Life Expectancy',
 'Life expectancy_95% CI - Low',
 'Life expectancy_95% CI - High',
 'Life expectancy_Life Expectancy (AIAN)',
 'Life expectancy_Life Expectancy (AIAN) 95% CI - Low',
 'Life expectancy_Life Expectancy (AIAN) 95% CI - High',
 'Life expectancy_Life Expectancy (Asian)',
 'Life expectancy_Life Expectancy (Asian) 95% CI - Low',
 'Life expectancy_Life Expectancy (Asian) 95% CI - High',
 'Life expectancy_Life Expectancy (Black)',
 'Life expectancy_Life Expectancy (Black) 95% CI - Low',
 'Life expectancy_Life Expectancy (Black) 95% CI - High',
 'Life expectancy_Life Expectancy (Hispanic)',
 'Life expectancy_Life Expectancy (Hispanic) 95% CI - Low',
 'Life expectancy_Life Expectancy (Hispanic) 95% CI - High',
 'Life expectancy_Life Expectancy (White)',
 'Life expectancy_Life Expectancy (White) 95% CI - Low',
 'Life expectancy_Life Expectancy (White) 95% CI - High',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN) 95% CI - Low',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (AIAN) 95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian) 95% CI - Low',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Asian) 95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Black) 95% CI - Low',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Black) 95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic) 95% CI - Low',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic) 95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (White) 95% CI - Low',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (White) 95% CI - High',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Child mortality_Child Mortality Rate (AIAN)',
 'Child mortality_Child Mortality Rate (AIAN) 95% CI - Low',
 'Child mortality_Child Mortality Rate (AIAN) 95% CI - High',
 'Child mortality_Child Mortality Rate (Asian)',
 'Child mortality_Child Mortality Rate (Asian) 95% CI - Low',
 'Child mortality_Child Mortality Rate (Asian) 95% CI - High',
 'Child mortality_Child Mortality Rate (Black)',
 'Child mortality_Child Mortality Rate (Black) 95% CI - Low',
 'Child mortality_Child Mortality Rate (Black) 95% CI - High',
 'Child mortality_Child Mortality Rate (Hispanic)',
 'Child mortality_Child Mortality Rate (Hispanic) 95% CI - Low',
 'Child mortality_Child Mortality Rate (Hispanic) 95% CI - High',
 'Child mortality_Child Mortality Rate (White)',
 'Child mortality_Child Mortality Rate (White) 95% CI - Low',
 'Child mortality_Child Mortality Rate (White) 95% CI - High',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Infant mortality_Infant Mortality Rate (AIAN)',
 'Infant mortality_Infant Mortality Rate (AIAN) 95% CI - Low',
 'Infant mortality_Infant Mortality Rate (AIAN) 95% CI - High',
 'Infant mortality_Infant Mortality Rate (Asian)',
 'Infant mortality_Infant Mortality Rate (Asian) 95% CI - Low',
 'Infant mortality_Infant Mortality Rate (Asian) 95% CI - High',
 'Infant mortality_Infant Mortality Rate (Black)',
 'Infant mortality_Infant Mortality Rate (Black) 95% CI - Low',
 'Infant mortality_Infant Mortality Rate (Black) 95% CI - High',
 'Infant mortality_Infant Mortality Rate (Hispanic)',
 'Infant mortality_Infant Mortality Rate (Hispanic) 95% CI - Low',
 'Infant mortality_Infant Mortality Rate (Hispanic) 95% CI - High',
 'Infant mortality_Infant Mortality Rate (White)',
 'Infant mortality_Infant Mortality Rate (White) 95% CI - Low',
 'Infant mortality_Infant Mortality Rate (White) 95% CI - High',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent physical distress_95% CI - Low',
 'Frequent physical distress_95% CI - High',
 'Frequent mental distress_% Frequent Mental Distress',
 'Frequent mental distress_95% CI - Low',
 'Frequent mental distress_95% CI - High',
 'Diabetes prevalence_95% CI - Low',
 'Diabetes prevalence_95% CI - High',
 'Drug overdose deaths_95% CI - Low',
 'Drug overdose deaths_95% CI - High',
 'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN)',
 'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN) 95% CI - Low',
 'Drug overdose deaths_Drug Overdose Mortality Rate (AIAN) 95% CI - High',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Asian)',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Asian) 95% CI - Low',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Asian) 95% CI - High',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Black)',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Black) 95% CI - Low',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Black) 95% CI - High',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic)',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic) 95% CI - Low',
 'Drug overdose deaths_Drug Overdose Mortality Rate (Hispanic) 95% CI - High',
 'Drug overdose deaths_Drug Overdose Mortality Rate (White)',
 'Drug overdose deaths_Drug Overdose Mortality Rate (White) 95% CI - Low',
 'Drug overdose deaths_Drug Overdose Mortality Rate (White) 95% CI - High',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Motor vehicle crash deaths_MV Mortality Rate (AIAN)',
 'Motor vehicle crash deaths_MV Mortality Rate (AIAN) 95% CI - Low',
 'Motor vehicle crash deaths_MV Mortality Rate (AIAN) 95% CI - High',
 'Motor vehicle crash deaths_MV Mortality Rate (Asian)',
 'Motor vehicle crash deaths_MV Mortality Rate (Asian) 95% CI - Low',
 'Motor vehicle crash deaths_MV Mortality Rate (Asian) 95% CI - High',
 'Motor vehicle crash deaths_MV Mortality Rate (Black)',
 'Motor vehicle crash deaths_MV Mortality Rate (Black) 95% CI - Low',
 'Motor vehicle crash deaths_MV Mortality Rate (Black) 95% CI - High',
 'Motor vehicle crash deaths_MV Mortality Rate (Hispanic)',
 'Motor vehicle crash deaths_MV Mortality Rate (Hispanic) 95% CI - Low',
 'Motor vehicle crash deaths_MV Mortality Rate (Hispanic) 95% CI - High',
 'Motor vehicle crash deaths_MV Mortality Rate (White)',
 'Motor vehicle crash deaths_MV Mortality Rate (White) 95% CI - Low',
 'Motor vehicle crash deaths_MV Mortality Rate (White) 95% CI - High',
 'Insufficient sleep_% Insufficient Sleep',
 'Insufficient sleep_95% CI - Low',
 'Insufficient sleep_95% CI - High',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Disconnected youth_% Disconnected Youth',
 'Disconnected youth_95% CI - Low',
 'Disconnected youth_95% CI - High',
 'Reading scores_Average Grade Performance',
 'Reading scores_Average Grade Performance (Asian)',
 'Reading scores_Average Grade Performance (Black)',
 'Reading scores_Average Grade Performance (Hispanic)',
 'Reading scores_Average Grade Performance (White)',
 'Math scores_Average Grade Performance',
 'Math scores_Average Grade Performance (Asian)',
 'Math scores_Average Grade Performance (Black)',
 'Math scores_Average Grade Performance (Hispanic)',
 'Math scores_Average Grade Performance (White)',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Median household income_Household Income (AIAN)',
 'Median household income_Household Income (AIAN) 95% CI - Low',
 'Median household income_Household Income (AIAN) 95% CI - High',
 'Median household income_Household Income (Asian)',
 'Median household income_Household Income (Asian) 95% CI - Low',
 'Median household income_Household Income (Asian) 95% CI - High',
 'Median household income_Household Income (Black)',
 'Median household income_Household Income (Black) 95% CI - Low',
 'Median household income_Household Income (Black) 95% CI - High',
 'Median household income_Household Income (Hispanic)',
 'Median household income_Household Income (Hispanic) 95% CI - Low',
 'Median household income_Household Income (Hispanic) 95% CI - High',
 'Median household income_Household Income (White)',
 'Median household income_Household Income (White) 95% CI - Low',
 'Median household income_Household Income (White) 95% CI - High',
 'Residential segregation - Black/White_Segregation index',
 'Residential segregation - non-White/White_Segregation Index',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High',
 'Homicides_Homicide Rate (AIAN)',
 'Homicides_Homicide Rate (AIAN) 95% CI - Low',
 'Homicides_Homicide Rate (AIAN) 95% CI - High',
 'Homicides_Homicide Rate (Asian)',
 'Homicides_Homicide Rate (Asian) 95% CI - Low',
 'Homicides_Homicide Rate (Asian) 95% CI - High',
 'Homicides_Homicide Rate (Black)',
 'Homicides_Homicide Rate (Black) 95% CI - Low',
 'Homicides_Homicide Rate (Black) 95% CI - High',
 'Homicides_Homicide Rate (Hispanic)',
 'Homicides_Homicide Rate (Hispanic) 95% CI - Low',
 'Homicides_Homicide Rate (Hispanic) 95% CI - High',
 'Homicides_Homicide Rate (White)',
 'Homicides_Homicide Rate (White) 95% CI - Low',
 'Homicides_Homicide Rate (White) 95% CI - High',
 'Suicides_# Deaths',
 'Suicides_Suicide Rate (Age-Adjusted)',
 'Suicides_95% CI - Low',
 'Suicides_95% CI - High',
 'Suicides_Crude Rate',
 'Suicides_Suicide Rate (AIAN)',
 'Suicides_Suicide Rate (AIAN) 95% CI - Low',
 'Suicides_Suicide Rate (AIAN) 95% CI - High',
 'Suicides_Suicide Rate (Asian)',
 'Suicides_Suicide Rate (Asian) 95% CI - Low',
 'Suicides_Suicide Rate (Asian) 95% CI - High',
 'Suicides_Suicide Rate (Black)',
 'Suicides_Suicide Rate (Black) 95% CI - Low',
 'Suicides_Suicide Rate (Black) 95% CI - High',
 'Suicides_Suicide Rate (Hispanic)',
 'Suicides_Suicide Rate (Hispanic) 95% CI - Low',
 'Suicides_Suicide Rate (Hispanic) 95% CI - High',
 'Suicides_Suicide Rate (White)',
 'Suicides_Suicide Rate (White) 95% CI - Low',
 'Suicides_Suicide Rate (White) 95% CI - High',
 'Firearm fatalities_# Firearm Fatalities',
 'Firearm fatalities_Firearm Fatalities Rate',
 'Firearm fatalities_95% CI - Low',
 'Firearm fatalities_95% CI - High',
 'Firearm fatalities_Firearm Fatalities Rate (AIAN)',
 'Firearm fatalities_Firearm Fatalities Rate (AIAN) 95% CI - Low',
 'Firearm fatalities_Firearm Fatalities Rate (AIAN) 95% CI - High',
 'Firearm fatalities_Firearm Fatalities Rate (Asian)',
 'Firearm fatalities_Firearm Fatalities Rate (Asian) 95% CI - Low',
 'Firearm fatalities_Firearm Fatalities Rate (Asian) 95% CI - High',
 'Firearm fatalities_Firearm Fatalities Rate (Black)',
 'Firearm fatalities_Firearm Fatalities Rate (Black) 95% CI - Low',
 'Firearm fatalities_Firearm Fatalities Rate (Black) 95% CI - High',
 'Firearm fatalities_Firearm Fatalities Rate (Hispanic)',
 'Firearm fatalities_Firearm Fatalities Rate (Hispanic) 95% CI - Low',
 'Firearm fatalities_Firearm Fatalities Rate (Hispanic) 95% CI - High',
 'Firearm fatalities_Firearm Fatalities Rate (White)',
 'Firearm fatalities_Firearm Fatalities Rate (White) 95% CI - Low',
 'Firearm fatalities_Firearm Fatalities Rate (White) 95% CI - High',
 'Juvenile arrests_Non-Petitioned Cases',
 'Juvenile arrests_Petitioned Cases',
 'Juvenile arrests_Denominator',
 'Juvenile arrests_Juvenile Arrest Rate',
 'Traffic volume_Average Traffic Volume per Meter of Major Roadways',
 'Homeownership_# Homeowners',
 'Homeownership_% Homeowners',
 'Homeownership_95% CI - Low',
 'Homeownership_95% CI - High',
 'Severe housing cost burden_# Households with Severe Cost Burden',
 'Severe housing cost burden_% Severe Housing Cost Burden',
 'Severe housing cost burden_95% CI - Low',
 'Severe housing cost burden_95% CI - High',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High']
                             ,axis=1)

In [37]:
df2.rename(columns = {'Unnamed: 0_level_0_FIPS':'FIPS',
 'Unnamed: 1_level_0_State':'State',
 'Unnamed: 2_level_0_County':'County',
 'Premature age-adjusted mortality_# Deaths':'Premature(under75)_Age_Adjusted_Mortality_#_Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Death Rate':'Premature(under75)_Age_Adjusted_Mortality_per_100K',
 'Child mortality_# Deaths':'Child(under18)_Mortality_#_Deaths',
 'Child mortality_Child Mortality Rate':'Child(under18)_Mortality_per_100K',
 'Infant mortality_# Deaths':'Infant(within1yr)_Mortality_#_Deaths',
 'Infant mortality_Infant Mortality Rate':'Infant(within1yr)_Mortality_per_1K_live_births',
 'Diabetes prevalence_% Adults with Diabetes':'%_Adults(20+)_Diagnosed_Diabetes',
 'HIV prevalence_# HIV Cases':'#_HIV_Cases',
 'HIV prevalence_HIV Prevalence Rate':'#_Ppl(Age13+)_diagnosed_HIV_per_100K',
 'Food insecurity_# Food Insecure':'#_Ppl_Food_Insecure',
 'Food insecurity_% Food Insecure':'%_Pop_w/o_Adequate_Access_to_Food',
 'Limited access to healthy foods_# Limited Access':'#_Ppl_Low_Income_or_Do_Not_Live_Near_Grocery_Store',
 'Limited access to healthy foods_% Limited Access to Healthy Foods':'%_Pop_with_Limited_Access_to_Food',
 'Drug overdose deaths_# Drug Overdose Deaths':'#_Drug_Overdose_Deaths',
 'Drug overdose deaths_Drug Overdose Mortality Rate':'Drug_Overdose_Deaths_per_100K',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths':'#_Motor_Vehicle_Deaths',
 'Motor vehicle crash deaths_Motor Vehicle Mortality Rate':'Motor_Vehicle_Deaths_per_100K',
 'Uninsured adults_# Uninsured':'#_Adults(under65)_w/o_Health_Insurance',
 'Uninsured adults_% Uninsured':'%_Adults(under65)_w/o_Health_Insurance',
 'Uninsured children_# Uninsured':'#_Children(under19)_w/o_Health_Insurance',
 'Uninsured children_% Uninsured':'%_Children(under19)_w/o_Health_Insurance',
 'Other primary care providers_Other Primary Care Provider Rate':'Other_Primary_Care_Providers(non_physician)_per_100K',
 'Other primary care providers_Other Primary Care Provider Ratio':'Pop_to_Other_Primary_Care_Providers(non_physician)',
 'Median household income_Median Household Income':'Median_Household_Income',
 'Children eligible for free or reduced price lunch_% Enrolled in Free or Reduced Lunch':'%_Children_Enrolled_in_Free/Reduced_Lunch',
 'Homicides_Homicide Rate':'#_Deaths_due_to_Homicide_per_100K',
 'Demographics_Population':'Resident_Population',
 'Demographics_% less than 18 years of age':'%_under_18_years_of_age',
 'Demographics_% 65 and over':'%_Age_65_and_over',
 'Demographics_# Black':'#_Non-Hispanic_Black_or_African_American',
 'Demographics_% Black':'%_Non-Hispanic_Black_or_African_American',
 'Demographics_# American Indian & Alaska Native':'#_American_Indian_or_Alaskan_Native',
 'Demographics_% American Indian & Alaska Native':'%_American_Indian_or_Alaskan_Native',
 'Demographics_# Asian':'#_Asian',
 'Demographics_% Asian':'%_Asian',
 'Demographics_# Native Hawaiian/Other Pacific Islander':'#_Native_Hawaiian/Other_Pacific_Islander',
 'Demographics_% Native Hawaiian/Other Pacific Islander':'%_Native_Hawaiian/Other_Pacific_Islander',
 'Demographics_# Hispanic':'#_Hispanic',
 'Demographics_% Hispanic':'%_Hispanic',
 'Demographics_# Non-Hispanic White':'#_Non-Hispanic_White',
 'Demographics_% Non-Hispanic White':'%_Non-Hispanic_White',
 'Demographics_# Not Proficient in English':'#_Not_Proficient_in_English',
 'Demographics_% Not Proficient in English':'%_Not_Proficient_in_English',
 'Demographics_% Female':'%_Female',
 'Demographics_# Rural':'#_Pop_Living_in_Rural_Area',
 'Demographics_% Rural':'%_Rural'}, 
           inplace = True)

In [38]:
df2.head()

,FIPS,State,County,Premature(under75)_Age_Adjusted_Mortality_#_Deaths,Premature(under75)_Age_Adjusted_Mortality_per_100K,Child(under18)_Mortality_#_Deaths,Child(under18)_Mortality_per_100K,Infant(within1yr)_Mortality_#_Deaths,Infant(within1yr)_Mortality_per_1K_live_births,%_Adults(20+)_Diagnosed_Diabetes,...,%_Native_Hawaiian/Other_Pacific_Islander,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural
0,48000,Texas,NaN,297317.0,340.057186,14722.0,50.296380,15745.0,5.714472,10.0,...,0.147492,11368849,39.610168,11912849,41.505516,1957944,7.563623,50.316413,3847522,15.300999
1,48001,Texas,Anderson,1130.0,568.866553,23.0,51.378278,NaN,NaN,10.4,...,0.158465,10430,17.965103,34059,58.664760,1388,2.529523,38.773964,39204,67.063533
2,48003,Texas,Andrews,180.0,372.435463,NaN,NaN,NaN,NaN,6.7,...,0.022065,10268,56.641659,7229,39.877538,1306,8.028524,48.841571,2440,16.502097
3,48005,Texas,Angelina,1375.0,461.769803,54.0,59.584234,50.0,5.950964,14.3,...,0.070041,19589,22.492307,52523,60.307491,3586,4.394823,51.214807,37383,43.082366
4,48007,Texas,Aransas,492.0,443.711298,NaN,NaN,NaN,NaN,18.7,...,0.092468,6526,27.429388,16063,67.514291,571,2.424011,50.609449,6313,27.260558


In [45]:
joined_df = pd.merge(df, df2, on='FIPS', how='outer')
joined_df.head()

,FIPS,State_x,County_x,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,%_Native_Hawaiian/Other_Pacific_Islander,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural
0,48000,Texas,NaN,297317.0,6650.619481,20.806624,3.716692,3.813413,NaN,8.328265,...,0.147492,11368849,39.610168,11912849,41.505516,1957944,7.563623,50.316413,3847522,15.300999
1,48001,Texas,Anderson,1130.0,10927.038043,19.727723,3.867507,3.971116,NaN,7.307041,...,0.158465,10430,17.965103,34059,58.664760,1388,2.529523,38.773964,39204,67.063533
2,48003,Texas,Andrews,180.0,7137.509901,19.851002,3.649546,3.731177,NaN,6.292750,...,0.022065,10268,56.641659,7229,39.877538,1306,8.028524,48.841571,2440,16.502097
3,48005,Texas,Angelina,1375.0,8913.812861,20.900430,4.091402,4.334904,NaN,9.050049,...,0.070041,19589,22.492307,52523,60.307491,3586,4.394823,51.214807,37383,43.082366
4,48007,Texas,Aransas,492.0,8770.440897,20.550541,4.116061,4.255922,NaN,9.263521,...,0.092468,6526,27.429388,16063,67.514291,571,2.424011,50.609449,6313,27.260558


In [46]:
list(joined_df.columns.values)

['FIPS',
 'State_x',
 'County_x',
 'Premature_Deaths',
 'Yrs_Potential_Life_Lost_per100K(YPLL)',
 '%_Fair/Poor_Health',
 'Avg_Days_Poor_Physical_Health_per_Month',
 'Avg_Days_Mentally_Unhealthy_per_Month',
 'Low_BirthWt_Unreliable',
 '%_Low_Birthweight_Births',
 '%_Adult_Smokers',
 '%_Adult_Obesity',
 'Healthy_Food_Access(0=Worst,10=Best)',
 '%_Adults_Physical_Inactivity',
 '%_Access_to_Excercise_Opportunities',
 '%_Adults_Excessive_Drinking',
 '#_Alcohol-Impaired_Motor_Vehicle_Deaths',
 '#_Motor_Vehicle_Deaths_x',
 '%_Driving_Deaths_with_Alcohol_Involvement',
 '#_Chlamydia_Cases',
 'Chlamydia_Cases_per_100K',
 'Teen_Birth_Rate',
 '#_Ppl_Under65_w/o_Insurance',
 '%_Ppl_Under65_w/o_Insurance',
 '#_Primary_Care_Physicians(PCP)_in_Pt_Care',
 'PCP_per_100K',
 'Population_to_PCP_ratio',
 '#_Dentists',
 'Dentists_per_100K',
 'Population_to_Dentists_ratio',
 '#_Mental_Health_Providers(MHP)',
 'MHP_per_100K',
 'Population_to_MHP_ratio',
 'Preventable_Hospitalization_Rate(per100K_Medicare_Enrol

In [47]:
joined_df = joined_df.drop(['State_y','County_y'],axis=1)
joined_df.rename(columns = {'State_x':'State','County_x':'County'},inplace = True)
joined_df.head()

,FIPS,State,County,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,%_Native_Hawaiian/Other_Pacific_Islander,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural
0,48000,Texas,NaN,297317.0,6650.619481,20.806624,3.716692,3.813413,NaN,8.328265,...,0.147492,11368849,39.610168,11912849,41.505516,1957944,7.563623,50.316413,3847522,15.300999
1,48001,Texas,Anderson,1130.0,10927.038043,19.727723,3.867507,3.971116,NaN,7.307041,...,0.158465,10430,17.965103,34059,58.664760,1388,2.529523,38.773964,39204,67.063533
2,48003,Texas,Andrews,180.0,7137.509901,19.851002,3.649546,3.731177,NaN,6.292750,...,0.022065,10268,56.641659,7229,39.877538,1306,8.028524,48.841571,2440,16.502097
3,48005,Texas,Angelina,1375.0,8913.812861,20.900430,4.091402,4.334904,NaN,9.050049,...,0.070041,19589,22.492307,52523,60.307491,3586,4.394823,51.214807,37383,43.082366
4,48007,Texas,Aransas,492.0,8770.440897,20.550541,4.116061,4.255922,NaN,9.263521,...,0.092468,6526,27.429388,16063,67.514291,571,2.424011,50.609449,6313,27.260558


In [48]:
joined_df.count()

FIPS                                                    255
State                                                   255
County                                                  254
Premature_Deaths                                        213
Yrs_Potential_Life_Lost_per100K(YPLL)                   213
%_Fair/Poor_Health                                      255
Avg_Days_Poor_Physical_Health_per_Month                 255
Avg_Days_Mentally_Unhealthy_per_Month                   255
Low_BirthWt_Unreliable                                   20
%_Low_Birthweight_Births                                236
%_Adult_Smokers                                         255
%_Adult_Obesity                                         255
Healthy_Food_Access(0=Worst,10=Best)                    254
%_Adults_Physical_Inactivity                            255
%_Access_to_Excercise_Opportunities                     255
%_Adults_Excessive_Drinking                             255
#_Alcohol-Impaired_Motor_Vehicle_Deaths 

In [49]:
joined_df.isnull().any()

FIPS                                                    False
State                                                   False
County                                                   True
Premature_Deaths                                         True
Yrs_Potential_Life_Lost_per100K(YPLL)                    True
%_Fair/Poor_Health                                      False
Avg_Days_Poor_Physical_Health_per_Month                 False
Avg_Days_Mentally_Unhealthy_per_Month                   False
Low_BirthWt_Unreliable                                   True
%_Low_Birthweight_Births                                 True
%_Adult_Smokers                                         False
%_Adult_Obesity                                         False
Healthy_Food_Access(0=Worst,10=Best)                     True
%_Adults_Physical_Inactivity                            False
%_Access_to_Excercise_Opportunities                     False
%_Adults_Excessive_Drinking                             False
#_Alcoho

In [50]:
joined_df.loc[df.isnull().any(axis=1)]

,FIPS,State,County,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,%_Native_Hawaiian/Other_Pacific_Islander,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural
0,48000,Texas,NaN,297317.0,6650.619481,20.806624,3.716692,3.813413,NaN,8.328265,...,0.147492,11368849,39.610168,11912849,41.505516,1957944,7.563623,50.316413,3847522,15.300999
1,48001,Texas,Anderson,1130.0,10927.038043,19.727723,3.867507,3.971116,NaN,7.307041,...,0.158465,10430,17.965103,34059,58.664760,1388,2.529523,38.773964,39204,67.063533
2,48003,Texas,Andrews,180.0,7137.509901,19.851002,3.649546,3.731177,NaN,6.292750,...,0.022065,10268,56.641659,7229,39.877538,1306,8.028524,48.841571,2440,16.502097
3,48005,Texas,Angelina,1375.0,8913.812861,20.900430,4.091402,4.334904,NaN,9.050049,...,0.070041,19589,22.492307,52523,60.307491,3586,4.394823,51.214807,37383,43.082366
4,48007,Texas,Aransas,492.0,8770.440897,20.550541,4.116061,4.255922,NaN,9.263521,...,0.092468,6526,27.429388,16063,67.514291,571,2.424011,50.609449,6313,27.260558
5,48009,Texas,Archer,125.0,8253.250524,13.891398,3.396114,3.833007,NaN,7.977737,...,0.034145,717,8.160710,7757,88.288186,79,0.945089,50.762577,8057,88.988292
6,48011,Texas,Armstrong,NaN,NaN,12.287990,3.075750,3.586202,x,8.609272,...,0.000000,151,7.980973,1685,89.059197,18,1.002786,51.479915,1901,100.000000
7,48013,Texas,Atascosa,611.0,7753.554711,23.678388,4.049933,4.002868,NaN,7.989096,...,0.127211,32529,64.657126,16669,33.132578,1839,4.064627,50.141125,27266,60.711184
8,48015,Texas,Austin,379.0,7371.154523,17.116889,3.594875,3.926105,NaN,7.949791,...,0.043349,8265,27.560105,18497,61.679282,1449,5.215044,50.135050,18853,66.344090
9,48017,Texas,Bailey,76.0,9061.655502,26.991708,4.491101,4.288898,NaN,7.457627,...,0.128077,4504,64.095631,2319,33.001281,881,13.346463,50.206347,2060,28.750872


In [53]:
joined_df = joined_df.fillna(value=0)
joined_df.head()

,FIPS,State,County,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,%_Native_Hawaiian/Other_Pacific_Islander,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural
0,48000,Texas,0,297317.0,6650.619481,20.806624,3.716692,3.813413,0,8.328265,...,0.147492,11368849,39.610168,11912849,41.505516,1957944,7.563623,50.316413,3847522,15.300999
1,48001,Texas,Anderson,1130.0,10927.038043,19.727723,3.867507,3.971116,0,7.307041,...,0.158465,10430,17.965103,34059,58.664760,1388,2.529523,38.773964,39204,67.063533
2,48003,Texas,Andrews,180.0,7137.509901,19.851002,3.649546,3.731177,0,6.292750,...,0.022065,10268,56.641659,7229,39.877538,1306,8.028524,48.841571,2440,16.502097
3,48005,Texas,Angelina,1375.0,8913.812861,20.900430,4.091402,4.334904,0,9.050049,...,0.070041,19589,22.492307,52523,60.307491,3586,4.394823,51.214807,37383,43.082366
4,48007,Texas,Aransas,492.0,8770.440897,20.550541,4.116061,4.255922,0,9.263521,...,0.092468,6526,27.429388,16063,67.514291,571,2.424011,50.609449,6313,27.260558


In [54]:
joined_df['Year']='2020'
joined_df.head()

,FIPS,State,County,Premature_Deaths,Yrs_Potential_Life_Lost_per100K(YPLL),%_Fair/Poor_Health,Avg_Days_Poor_Physical_Health_per_Month,Avg_Days_Mentally_Unhealthy_per_Month,Low_BirthWt_Unreliable,%_Low_Birthweight_Births,...,#_Hispanic,%_Hispanic,#_Non-Hispanic_White,%_Non-Hispanic_White,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Pop_Living_in_Rural_Area,%_Rural,Year
0,48000,Texas,0,297317.0,6650.619481,20.806624,3.716692,3.813413,0,8.328265,...,11368849,39.610168,11912849,41.505516,1957944,7.563623,50.316413,3847522,15.300999,2020
1,48001,Texas,Anderson,1130.0,10927.038043,19.727723,3.867507,3.971116,0,7.307041,...,10430,17.965103,34059,58.664760,1388,2.529523,38.773964,39204,67.063533,2020
2,48003,Texas,Andrews,180.0,7137.509901,19.851002,3.649546,3.731177,0,6.292750,...,10268,56.641659,7229,39.877538,1306,8.028524,48.841571,2440,16.502097,2020
3,48005,Texas,Angelina,1375.0,8913.812861,20.900430,4.091402,4.334904,0,9.050049,...,19589,22.492307,52523,60.307491,3586,4.394823,51.214807,37383,43.082366,2020
4,48007,Texas,Aransas,492.0,8770.440897,20.550541,4.116061,4.255922,0,9.263521,...,6526,27.429388,16063,67.514291,571,2.424011,50.609449,6313,27.260558,2020


In [55]:
joined_df.to_csv(os.path.join("Cleaned_Files","Texas_County_Health_Ranking_Test.csv"),index=False)